# Các trường hợp sử dụng phức tạp của Agent với Claude

## Giới thiệu

Trong bài học này, chúng ta sẽ khám phá các ứng dụng nâng cao của Agent trong LangChain, bao gồm:

1. **Custom Tools**: Tạo các công cụ tùy chỉnh cho nhu cầu cụ thể
2. **Multi-step reasoning**: Agent giải quyết vấn đề phức tạp qua nhiều bước
3. **Tool combinations**: Kết hợp nhiều công cụ để giải quyết vấn đề
4. **Error handling**: Xử lý lỗi và retry strategies

### Các loại Tools chúng ta sẽ sử dụng:

- **Search Tools**: Google Search, DuckDuckGo, Arxiv
- **Computation Tools**: Calculator, Python REPL
- **Information Tools**: Wikipedia, Weather API
- **Custom Tools**: Database query, File operations, API calls

## Cài đặt và Import

In [6]:
# Cài đặt các thư viện cần thiết
pip install langchain langchain-anthropic langchain-community
pip install arxiv wikipedia-api duckduckgo-search google-search-results
pip install python-dotenv requests beautifulsoup4 dotenv

In [11]:
import os
from dotenv import load_dotenv
from typing import Optional, Type, List, Dict, Any
from datetime import datetime
import json
import requests
from pydantic import BaseModel, Field

# LangChain imports
from langchain_anthropic import ChatAnthropic
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler

# Community tools
from langchain_community.tools import (
    DuckDuckGoSearchRun,
    WikipediaQueryRun,
    ArxivQueryRun
)
from langchain_community.utilities import (
    DuckDuckGoSearchAPIWrapper,
    WikipediaAPIWrapper,
    ArxivAPIWrapper
)

# Load environment variables
load_dotenv()

# Khởi tạo Claude
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

print("✅ Đã khởi tạo Claude và các thư viện thành công!")

ModuleNotFoundError: No module named 'dotenv'

## 1. Tạo Custom Tools

Custom tools cho phép chúng ta mở rộng khả năng của agent theo nhu cầu cụ thể.

In [ ]:
# Custom Tool 1: Stock Price Checker (giả lập)
@tool
def get_stock_price(symbol: str) -> str:
    """
    Lấy giá cổ phiếu hiện tại của một mã chứng khoán.
    Args:
        symbol: Mã cổ phiếu (ví dụ: AAPL, GOOGL, MSFT)
    """
    # Giả lập API call - trong thực tế sẽ gọi API thật
    mock_prices = {
        "AAPL": 189.95,
        "GOOGL": 141.80,
        "MSFT": 378.85,
        "TSLA": 242.64,
        "AMZN": 155.33
    }
    
    price = mock_prices.get(symbol.upper())
    if price:
        return f"Giá hiện tại của {symbol.upper()}: ${price} USD"
    else:
        return f"Không tìm thấy giá cho mã {symbol}"

# Custom Tool 2: Currency Converter
@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Chuyển đổi tiền tệ từ một loại sang loại khác.
    Args:
        amount: Số tiền cần chuyển đổi
        from_currency: Mã tiền tệ gốc (USD, EUR, VND, etc.)
        to_currency: Mã tiền tệ đích
    """
    # Giả lập tỷ giá - trong thực tế sẽ gọi API
    exchange_rates = {
        "USD": {"EUR": 0.92, "VND": 24350, "GBP": 0.79},
        "EUR": {"USD": 1.09, "VND": 26500, "GBP": 0.86},
        "VND": {"USD": 0.000041, "EUR": 0.000038, "GBP": 0.000032}
    }
    
    from_curr = from_currency.upper()
    to_curr = to_currency.upper()
    
    if from_curr == to_curr:
        return f"{amount} {from_curr} = {amount} {to_curr}"
    
    if from_curr in exchange_rates and to_curr in exchange_rates[from_curr]:
        rate = exchange_rates[from_curr][to_curr]
        converted = amount * rate
        return f"{amount} {from_curr} = {converted:.2f} {to_curr} (Tỷ giá: 1 {from_curr} = {rate} {to_curr})"
    
    return f"Không thể chuyển đổi từ {from_curr} sang {to_curr}"

# Custom Tool 3: Database Query Tool
class DatabaseQueryInput(BaseModel):
    query_type: str = Field(description="Loại query: 'customer', 'product', 'order'")
    filter_key: str = Field(description="Key để filter (id, name, status, etc.)")
    filter_value: str = Field(description="Giá trị để filter")

def database_query(query_type: str, filter_key: str, filter_value: str) -> str:
    """
    Truy vấn database giả lập để lấy thông tin.
    """
    # Mock database
    mock_db = {
        "customer": [
            {"id": "C001", "name": "Nguyễn Văn A", "status": "active", "credit": 5000000},
            {"id": "C002", "name": "Trần Thị B", "status": "active", "credit": 3000000},
            {"id": "C003", "name": "Lê Văn C", "status": "inactive", "credit": 0}
        ],
        "product": [
            {"id": "P001", "name": "Laptop Dell", "price": 15000000, "stock": 25},
            {"id": "P002", "name": "iPhone 15", "price": 25000000, "stock": 10},
            {"id": "P003", "name": "Samsung TV", "price": 20000000, "stock": 5}
        ],
        "order": [
            {"id": "O001", "customer_id": "C001", "total": 40000000, "status": "completed"},
            {"id": "O002", "customer_id": "C002", "total": 15000000, "status": "pending"},
            {"id": "O003", "customer_id": "C001", "total": 25000000, "status": "processing"}
        ]
    }
    
    if query_type not in mock_db:
        return f"Loại query '{query_type}' không hợp lệ"
    
    results = []
    for record in mock_db[query_type]:
        if str(record.get(filter_key, "")).lower() == str(filter_value).lower():
            results.append(record)
    
    if results:
        return f"Tìm thấy {len(results)} kết quả:\n{json.dumps(results, ensure_ascii=False, indent=2)}"
    else:
        return f"Không tìm thấy {query_type} với {filter_key}='{filter_value}'"

database_tool = StructuredTool.from_function(
    func=database_query,
    name="database_query",
    description="Truy vấn database để lấy thông tin về customer, product, hoặc order",
    args_schema=DatabaseQueryInput
)

print("✅ Đã tạo xong các custom tools!")

## 2. Khởi tạo các Search và Information Tools

In [ ]:
# DuckDuckGo Search
search = DuckDuckGoSearchRun(
    api_wrapper=DuckDuckGoSearchAPIWrapper(max_results=3)
)

# Wikipedia
wikipedia = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
)

# Arxiv
arxiv = ArxivQueryRun(
    api_wrapper=ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=1000)
)

# Calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Tính toán biểu thức toán học.
    Args:
        expression: Biểu thức toán học (ví dụ: "2 + 2", "10 * 5", "sqrt(16)")
    """
    try:
        # Import math functions
        import math
        
        # Create safe namespace
        safe_dict = {
            'sqrt': math.sqrt,
            'pow': math.pow,
            'sin': math.sin,
            'cos': math.cos,
            'tan': math.tan,
            'log': math.log,
            'pi': math.pi,
            'e': math.e
        }
        
        # Evaluate expression
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"Kết quả: {result}"
    except Exception as e:
        return f"Lỗi khi tính toán: {str(e)}"

print("✅ Đã khởi tạo xong các search và information tools!")

## 3. Tạo Agent với nhiều Tools

Bây giờ chúng ta sẽ tạo một agent mạnh mẽ với tất cả các tools đã định nghĩa.

In [ ]:
# Tập hợp tất cả tools
tools = [
    # Custom tools
    get_stock_price,
    convert_currency,
    database_tool,
    calculator,
    
    # Search tools
    Tool(
        name="search",
        func=search.run,
        description="Tìm kiếm thông tin trên internet. Sử dụng khi cần thông tin mới nhất hoặc thông tin chung."
    ),
    Tool(
        name="wikipedia",
        func=wikipedia.run,
        description="Tìm kiếm thông tin trên Wikipedia. Sử dụng cho kiến thức bách khoa, lịch sử, khoa học."
    ),
    Tool(
        name="arxiv",
        func=arxiv.run,
        description="Tìm kiếm papers khoa học trên Arxiv. Sử dụng cho nghiên cứu học thuật, AI/ML papers."
    )
]

# Memory cho agent
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,  # Nhớ 5 trao đổi gần nhất
    return_messages=True
)

# Custom callback để theo dõi quá trình suy nghĩ
class VerboseCallbackHandler(BaseCallbackHandler):
    def on_tool_start(self, serialized: Dict[str, Any], input_str: str, **kwargs) -> None:
        tool_name = serialized.get('name', 'Unknown')
        print(f"\n🔧 Đang sử dụng tool: {tool_name}")
        print(f"   Input: {input_str}")
    
    def on_tool_end(self, output: str, **kwargs) -> None:
        print(f"   Output: {output[:200]}..." if len(output) > 200 else f"   Output: {output}")

# Tạo agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    callbacks=[VerboseCallbackHandler()]
)

print("✅ Đã tạo agent với đầy đủ tools!")
print(f"\n📋 Danh sách tools có sẵn:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description[:60]}...")

## 4. Use Case 1: Phân tích đầu tư đa nguồn

Agent sẽ thu thập thông tin từ nhiều nguồn để phân tích một cơ hội đầu tư.

In [ ]:
print("🎯 USE CASE 1: PHÂN TÍCH ĐẦU TƯ")
print("=" * 60)

investment_query = """
Tôi đang cân nhắc đầu tư vào Apple (AAPL). Hãy giúp tôi:
1. Kiểm tra giá cổ phiếu hiện tại
2. Tìm hiểu về tình hình kinh doanh gần đây của Apple
3. So sánh với giá của Microsoft (MSFT)
4. Nếu tôi có $10,000 USD, tôi có thể mua được bao nhiêu cổ phiếu của mỗi công ty?
5. Chuyển đổi $10,000 USD sang VND để tôi biết số tiền tương đương
"""

response = agent.run(investment_query)
print("\n💡 KẾT QUẢ PHÂN TÍCH:")
print("=" * 60)
print(response)

## 5. Use Case 2: Nghiên cứu học thuật phức tạp

Agent sẽ tìm kiếm và tổng hợp thông tin từ nhiều nguồn học thuật.

In [ ]:
print("\n🎯 USE CASE 2: NGHIÊN CỨU HỌC THUẬT")
print("=" * 60)

research_query = """
Tôi đang nghiên cứu về Transformer architecture trong NLP. Hãy giúp tôi:
1. Tìm paper gốc về Transformer trên Arxiv
2. Giải thích ngắn gọn về Transformer từ Wikipedia
3. Tìm kiếm các ứng dụng mới nhất của Transformer trong năm 2024
4. Tính toán: Nếu một Transformer model có 12 layers, mỗi layer có 768 hidden units, 
   tổng số parameters là bao nhiêu? (giả sử mỗi unit cần 4 parameters)
"""

response = agent.run(research_query)
print("\n📚 KẾT QUẢ NGHIÊN CỨU:")
print("=" * 60)
print(response)

## 6. Use Case 3: Customer Service phức tạp

Agent xử lý yêu cầu khách hàng cần truy vấn database và tính toán.

In [ ]:
print("\n🎯 USE CASE 3: CUSTOMER SERVICE")
print("=" * 60)

customer_query = """
Tôi là khách hàng có ID là C001. Hãy giúp tôi:
1. Kiểm tra thông tin tài khoản của tôi
2. Xem tất cả đơn hàng của tôi 
3. Tính tổng giá trị các đơn hàng
4. Kiểm tra sản phẩm P002 còn hàng không và giá bao nhiêu
5. Nếu tôi mua 2 sản phẩm P002, tổng tiền là bao nhiêu và còn lại bao nhiêu credit?
"""

response = agent.run(customer_query)
print("\n🛍️ KẾT QUẢ XỬ LÝ:")
print("=" * 60)
print(response)

## 7. Custom Agent với ReAct Prompt tùy chỉnh

Tạo agent với prompt ReAct được tùy chỉnh cho tiếng Việt.

In [ ]:
# Custom ReAct prompt cho tiếng Việt
vietnamese_react_prompt = PromptTemplate.from_template("""
Bạn là một AI assistant thông minh có khả năng sử dụng nhiều công cụ để giúp người dùng.

Bạn có quyền truy cập vào các công cụ sau:
{tools}

Để sử dụng công cụ, hãy tuân theo format:
Suy nghĩ: Tôi cần làm gì để trả lời câu hỏi này
Hành động: tên_công_cụ
Đầu vào: {{"key": "value"}}
Quan sát: kết quả từ công cụ

... (quá trình này có thể lặp lại nhiều lần)

Suy nghĩ: Bây giờ tôi đã có đủ thông tin để trả lời
Câu trả lời cuối cùng: câu trả lời chi tiết cho người dùng

Lịch sử hội thoại:
{chat_history}

Câu hỏi: {input}
Hãy suy nghĩ từng bước và sử dụng công cụ khi cần thiết.
{agent_scratchpad}
""")

# Tạo custom agent với Vietnamese prompt
custom_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=vietnamese_react_prompt
)

# Agent executor với custom agent
vietnamese_agent = AgentExecutor(
    agent=custom_agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✅ Đã tạo Vietnamese ReAct Agent!")

In [ ]:
# Test Vietnamese agent
print("\n🎯 TEST VIETNAMESE AGENT")
print("=" * 60)

vietnamese_query = """
Giúp tôi lập kế hoạch du lịch:
1. Tìm thông tin về thành phố Paris trên Wikipedia
2. Chuyển đổi 2000 EUR sang VND để biết ngân sách
3. Tính xem với 50 triệu VND, tôi có được bao nhiêu EUR?
"""

response = vietnamese_agent.invoke({"input": vietnamese_query})
print("\n✈️ KẾ HOẠCH DU LỊCH:")
print("=" * 60)
print(response["output"])

## 8. Agent với Error Handling và Retry Logic

Xây dựng agent có khả năng xử lý lỗi và thử lại khi gặp vấn đề.

In [ ]:
# Tool có thể gặp lỗi
@tool
def unstable_api_call(query: str) -> str:
    """
    Gọi một API không ổn định (giả lập). 
    Có 50% khả năng thất bại.
    """
    import random
    
    if random.random() < 0.5:
        raise Exception("API tạm thời không khả dụng")
    
    return f"Kết quả từ API cho query '{query}': Thành công!"

# Wrapper với retry logic
class RetryTool(BaseTool):
    name = "retry_api_call"
    description = "Gọi API với khả năng retry khi thất bại"
    
    def _run(self, query: str, max_retries: int = 3) -> str:
        for attempt in range(max_retries):
            try:
                result = unstable_api_call.run(query)
                return f"✅ {result} (sau {attempt + 1} lần thử)"
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"❌ Lần thử {attempt + 1} thất bại: {e}. Đang thử lại...")
                else:
                    return f"❌ Thất bại sau {max_retries} lần thử. Lỗi: {e}"
        return "Không thể hoàn thành request"
    
    async def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not implemented")

# Thêm retry tool vào danh sách
tools_with_retry = tools + [RetryTool(), unstable_api_call]

# Agent với error handling
robust_agent = initialize_agent(
    tools=tools_with_retry,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=15,  # Tăng số iterations cho retry
    early_stopping_method="generate"  # Generate response khi hết iterations
)

print("✅ Đã tạo Robust Agent với error handling!")

In [ ]:
# Test robust agent
print("\n🎯 TEST ROBUST AGENT")
print("=" * 60)

robust_query = """
Hãy thực hiện các task sau:
1. Gọi unstable API với query "test data" 
2. Nếu thất bại, hãy thử dùng retry_api_call
3. Tính 15% của 8500
4. Tìm thông tin về error handling trong programming
"""

response = robust_agent.run(robust_query)
print("\n🛡️ KẾT QUẢ XỬ LÝ:")
print("=" * 60)
print(response)

## 9. Multi-Agent System

Tạo hệ thống nhiều agent chuyên biệt làm việc cùng nhau.

In [ ]:
# Research Agent - Chuyên tìm kiếm và nghiên cứu
research_tools = [
    Tool(name="search", func=search.run, description="Tìm kiếm internet"),
    Tool(name="wikipedia", func=wikipedia.run, description="Tìm kiếm Wikipedia"),
    Tool(name="arxiv", func=arxiv.run, description="Tìm papers khoa học")
]

research_agent = initialize_agent(
    tools=research_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# Finance Agent - Chuyên về tài chính
finance_tools = [
    get_stock_price,
    convert_currency,
    calculator
]

finance_agent = initialize_agent(
    tools=finance_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# Data Agent - Chuyên xử lý dữ liệu
data_tools = [
    database_tool,
    calculator
]

data_agent = initialize_agent(
    tools=data_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# Coordinator Agent - Điều phối các agent khác
@tool
def delegate_to_research(query: str) -> str:
    """Giao nhiệm vụ cho Research Agent"""
    return research_agent.run(query)

@tool
def delegate_to_finance(query: str) -> str:
    """Giao nhiệm vụ cho Finance Agent"""
    return finance_agent.run(query)

@tool
def delegate_to_data(query: str) -> str:
    """Giao nhiệm vụ cho Data Agent"""
    return data_agent.run(query)

coordinator_tools = [
    delegate_to_research,
    delegate_to_finance,
    delegate_to_data
]

coordinator = initialize_agent(
    tools=coordinator_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

print("✅ Đã tạo Multi-Agent System!")
print("\n👥 Các agent trong hệ thống:")
print("   - 🔍 Research Agent: Tìm kiếm và nghiên cứu")
print("   - 💰 Finance Agent: Phân tích tài chính")
print("   - 📊 Data Agent: Xử lý dữ liệu")
print("   - 🎯 Coordinator: Điều phối các agent")

In [ ]:
# Test Multi-Agent System
print("\n🎯 TEST MULTI-AGENT SYSTEM")
print("=" * 60)

complex_query = """
Tôi cần phân tích toàn diện về Tesla:
1. Tìm hiểu lịch sử và thông tin cơ bản về công ty Tesla
2. Kiểm tra giá cổ phiếu TSLA hiện tại
3. Tìm nghiên cứu mới nhất về xe điện tự lái
4. Nếu tôi có 100 triệu VND, có thể mua được bao nhiêu cổ phiếu TSLA?
"""

print("🎯 Coordinator đang phân công nhiệm vụ...\n")
response = coordinator.run(complex_query)

print("\n📋 BÁO CÁO TỔNG HỢP:")
print("=" * 60)
print(response)

## 10. Agent với Planning và Reasoning phức tạp

Tạo agent có khả năng lập kế hoạch nhiều bước cho các task phức tạp.

In [ ]:
# Planning prompt
planning_prompt = PromptTemplate.from_template("""
Bạn là một AI planner xuất sắc. Nhiệm vụ của bạn là:
1. Phân tích yêu cầu của người dùng
2. Chia nhỏ thành các bước cụ thể
3. Xác định tools cần thiết cho mỗi bước
4. Thực hiện từng bước một cách có hệ thống

Công cụ có sẵn:
{tools}

Format suy nghĩ:
📋 KẾ HOẠCH:
- Bước 1: [mô tả] → Tool: [tool_name]
- Bước 2: [mô tả] → Tool: [tool_name]
...

Sau đó thực hiện từng bước:
🔄 THỰC HIỆN BƯỚC X:
Hành động: [tool_name]
Đầu vào: [input]
Quan sát: [result]

Lịch sử: {chat_history}
Yêu cầu: {input}
{agent_scratchpad}
""")

# Create planning agent
planning_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=planning_prompt
)

planner = AgentExecutor(
    agent=planning_agent,
    tools=tools,
    memory=ConversationBufferWindowMemory(k=10),
    verbose=True,
    max_iterations=20
)

print("✅ Đã tạo Planning Agent!")

In [ ]:
# Test planning agent với task phức tạp
print("\n🎯 TEST PLANNING AGENT")
print("=" * 60)

planning_task = """
Giúp tôi lên kế hoạch mở một quán cà phê:

1. Tìm hiểu về ngành kinh doanh cà phê
2. Ước tính vốn cần thiết (tính theo USD và VND)
3. Kiểm tra trong database xem có khách hàng nào status 'active' để làm khách hàng tiềm năng
4. Tính toán: Nếu bán 100 ly cà phê/ngày, giá 50,000 VND/ly, doanh thu tháng là bao nhiêu?
5. Tìm research về xu hướng cà phê specialty
"""

response = planner.invoke({"input": planning_task})

print("\n☕ KẾ HOẠCH KINH DOANH:")
print("=" * 60)
print(response["output"])

## Best Practices và Production Tips

### 1. Tool Design Best Practices

In [ ]:
# Best Practice: Tool với validation và error handling
class RobustTool(BaseTool):
    name = "robust_tool_example"
    description = "Tool mẫu với đầy đủ validation và error handling"
    
    def _validate_input(self, input_data: Dict[str, Any]) -> bool:
        """Validate input trước khi xử lý"""
        required_fields = ["query", "type"]
        for field in required_fields:
            if field not in input_data:
                raise ValueError(f"Missing required field: {field}")
        
        if input_data["type"] not in ["search", "analyze", "compute"]:
            raise ValueError(f"Invalid type: {input_data['type']}")
        
        return True
    
    def _run(self, input_str: str) -> str:
        try:
            # Parse input
            input_data = json.loads(input_str)
            
            # Validate
            self._validate_input(input_data)
            
            # Process based on type
            if input_data["type"] == "search":
                return f"Searched for: {input_data['query']}"
            elif input_data["type"] == "analyze":
                return f"Analyzed: {input_data['query']}"
            elif input_data["type"] == "compute":
                return f"Computed: {input_data['query']}"
            
        except json.JSONDecodeError:
            return "Error: Invalid JSON format. Please provide valid JSON."
        except ValueError as e:
            return f"Validation Error: {str(e)}"
        except Exception as e:
            return f"Unexpected Error: {str(e)}"
    
    async def _arun(self, input_str: str) -> str:
        # Implement async version if needed
        raise NotImplementedError("Async not implemented")

print("✅ Robust Tool Pattern created!")

### 2. Performance Monitoring

In [ ]:
import time
from typing import Dict, List

class PerformanceMonitor:
    def __init__(self):
        self.metrics = {
            "tool_calls": {},
            "total_time": 0,
            "errors": []
        }
    
    def record_tool_call(self, tool_name: str, duration: float, success: bool):
        if tool_name not in self.metrics["tool_calls"]:
            self.metrics["tool_calls"][tool_name] = {
                "count": 0,
                "total_time": 0,
                "failures": 0
            }
        
        self.metrics["tool_calls"][tool_name]["count"] += 1
        self.metrics["tool_calls"][tool_name]["total_time"] += duration
        if not success:
            self.metrics["tool_calls"][tool_name]["failures"] += 1
    
    def get_report(self) -> str:
        report = "\n📊 PERFORMANCE REPORT\n" + "=" * 50 + "\n"
        
        for tool, stats in self.metrics["tool_calls"].items():
            avg_time = stats["total_time"] / stats["count"] if stats["count"] > 0 else 0
            success_rate = (stats["count"] - stats["failures"]) / stats["count"] * 100 if stats["count"] > 0 else 0
            
            report += f"\n🔧 {tool}:\n"
            report += f"   Calls: {stats['count']}\n"
            report += f"   Avg Time: {avg_time:.2f}s\n"
            report += f"   Success Rate: {success_rate:.1f}%\n"
        
        return report

# Monitored tool wrapper
def create_monitored_tool(tool: Tool, monitor: PerformanceMonitor) -> Tool:
    original_func = tool.func
    
    def monitored_func(*args, **kwargs):
        start_time = time.time()
        success = True
        
        try:
            result = original_func(*args, **kwargs)
            return result
        except Exception as e:
            success = False
            raise e
        finally:
            duration = time.time() - start_time
            monitor.record_tool_call(tool.name, duration, success)
    
    return Tool(
        name=tool.name,
        func=monitored_func,
        description=tool.description
    )

# Example usage
monitor = PerformanceMonitor()
print("✅ Performance monitoring system created!")

### 3. Agent Caching Strategy

In [ ]:
from functools import lru_cache
import hashlib

class CachedAgent:
    def __init__(self, agent: AgentExecutor, cache_size: int = 100):
        self.agent = agent
        self.cache_size = cache_size
        self._cache = {}
    
    def _get_cache_key(self, query: str) -> str:
        """Generate cache key from query"""
        return hashlib.md5(query.encode()).hexdigest()
    
    def run(self, query: str, use_cache: bool = True) -> str:
        cache_key = self._get_cache_key(query)
        
        # Check cache
        if use_cache and cache_key in self._cache:
            print("💾 Cache hit! Returning cached result.")
            return self._cache[cache_key]
        
        # Run agent
        print("🔄 Cache miss. Running agent...")
        result = self.agent.run(query)
        
        # Update cache
        if len(self._cache) >= self.cache_size:
            # Remove oldest entry (simple FIFO)
            oldest_key = next(iter(self._cache))
            del self._cache[oldest_key]
        
        self._cache[cache_key] = result
        return result
    
    def clear_cache(self):
        """Clear all cached results"""
        self._cache.clear()
        print("🗑️ Cache cleared!")

# Create cached agent
cached_agent = CachedAgent(agent, cache_size=50)
print("✅ Caching system created!")

## Tổng kết

### Các Use Cases đã cover:

1. **Multi-source Analysis**: Agent thu thập và phân tích thông tin từ nhiều nguồn
2. **Academic Research**: Kết hợp Arxiv, Wikipedia và search để nghiên cứu
3. **Customer Service**: Xử lý yêu cầu phức tạp với database queries
4. **Multi-Agent Systems**: Nhiều agent chuyên biệt làm việc cùng nhau
5. **Planning & Reasoning**: Agent lập kế hoạch chi tiết cho tasks phức tạp

### Best Practices:

1. **Tool Design**:
   - Validation đầu vào kỹ lưỡng
   - Error handling cho mọi trường hợp
   - Descriptions rõ ràng và chi tiết

2. **Agent Configuration**:
   - Sử dụng memory cho context
   - Set max_iterations phù hợp
   - Handle parsing errors

3. **Performance**:
   - Implement caching khi có thể
   - Monitor tool usage
   - Optimize tool calls

4. **Production Considerations**:
   - Rate limiting cho external APIs
   - Timeout handling
   - Logging và monitoring
   - Security cho sensitive operations

### Mở rộng:

- Tích hợp với các APIs thực tế
- Implement async operations
- Add streaming responses
- Build agent orchestration systems
- Integrate với workflow engines